---
title: "Predicting Chronic Kidney Disease"
subtitle: "with Machine Learning & Visual Analytics"
author: "Shreemithra Naveen and Sai Laasya Gorantla"
date: "April 2025"
title-slide-attributes:
      data-background-image: images/background_image1.png
      data-background-size: cover
      data-background-opacity: "0.8"
theme: serif
format:
  revealjs:    
    slide-number: true
    transition: slide
    background-transition: slide
jupyter: python3
editor: visual
execute:
  warning: false
  message: false
  echo: false
---

In [ ]:
#| label: load-packages
#| include: false

#Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#| label: load-data
#| include: false
disease = pd.read_csv('kidney_disease.csv')

#Dropping off the column 'id'
disease = disease.drop('id', axis = 1)

#Renaming the columns for easy understanding the column and dataset clearly
disease.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell',
'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea', 'serum_creatinine', 'sodium','potassium', 'haemoglobin', 'packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count',
'hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 'appetite', 'peda_edema','aanemia', 'class']

#Converting the column, the errors is applied for not considering the NA values
disease[['packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count']] = disease[['packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count']].apply(pd.to_numeric, errors = 'coerce')

#As the 6-7 columns has just 1,2 or 4 missing values
disease.dropna(subset=['hypertension', 'diabetes_mellitus','coronary_artery_disease','appetite', 'peda_edema','aanemia', 'bacteria', 'pus_cell_clumps'], inplace = True)

#disease.isna().sum()

categorical_null = ['red_blood_cells', 'pus_cell']

values = ['normal', 'abnormal']
for feature in categorical_null:
    print(disease[feature].unique(), disease[feature].isna().sum())
    disease[feature].fillna(random.choice(values), inplace=True)

categorical = disease.select_dtypes(include = 'object')
categorical.columns 

## Replacing the incorrect values - cleaning

# Clean 'diabetes_mellitus' column
disease['diabetes_mellitus']= disease['diabetes_mellitus'].replace(to_replace={'\tno': 'no', '\tyes': 'yes', ' yes': 'yes'})
disease['coronary_artery_disease'] = disease['coronary_artery_disease'].replace(to_replace='\tno',value='no')
disease['class'] = disease['class'].replace(to_replace={'ckd\t': 'ckd'})

#Converting the class to the numerical column and also assigning binary values
disease['class'] = disease['class'].map({'ckd': 0, 'notckd': 1})
disease['class'] = pd.to_numeric(disease['class'], errors='coerce')

#plt.figure(figsize=(15,15))
#for i,col_name in enumerate(categorical.columns, start = 1):
#    plt.subplot(5,5,i)
#    sns.countplot(x = disease[col_name])
#plt.tight_layout()
#plt.show()

#Fetching the numerical columns
numerical_columns = disease.select_dtypes(exclude = 'object')

#plt.figure(figsize=(15,15))
#sns.pairplot(numerical_columns)
#plt.show()

In [ ]:
#| include: false
#| eval: false

pairs = [
    ('haemoglobin', 'packed_cell_volume'),
    ('packed_cell_volume', 'red_blood_cell_count'),
    ('haemoglobin', 'red_blood_cell_count'),
    ('sugar', 'blood_glucose_random'),
    ('blood_urea', 'red_blood_cell_count'),
    ('blood_urea', 'haemoglobin'),
    ('albumin', 'haemoglobin'),
    ('serum_creatinine', 'sodium')
]

# Set a polished theme
sns.set_theme(style="whitegrid", palette="viridis")

# Number of plots
n = len(pairs)
rows = (n + 1) // 2

# Create subplot grid
plt.figure(figsize=(18, rows * 4))
for i, (x, y) in enumerate(pairs, 1):
    plt.subplot(rows, 3, i)
    
    # Only plot if both columns exist and have no missing values
    if x in disease.columns and y in disease.columns:
        sns.scatterplot(data=disease, x=x, y=y, hue='class', palette='Set1', alpha=0.8, s=60)
        sns.regplot(data=disease, x=x, y=y, scatter=False, color='black', line_kws={"linewidth": 2, "linestyle": "--"})
        
        # Compute correlation for title
        corr_val = disease[[x, y]].dropna().corr().iloc[0, 1]
        plt.title(f"{x} vs {y} (r = {corr_val:.2f})", fontsize=12)
        plt.xlabel(x)
        plt.ylabel(y)

plt.suptitle('CKD Dataset – Strongly Correlated Feature Pairs with Class Highlighting', fontsize=20, y=1.02)

plt.tight_layout()
plt.show()

## Project Overview {.center}

- Over **850 million people worldwide** have kidney diseases — more than diabetes and cancer combined.
- **1 in 10 people globally** are affected by CKD.
- CKD is the **8th leading cause of death** worldwide (WHO, 2020).

<span style="font-size: 0.65em; color: #555;">
Source: World Health Organization (WHO), 2020 – <a href="https://www.who.int/news-room/fact-sheets/detail/kidney-disease">Global Facts on CKD</a>
</span>


# Silent & Dangerous

------------------------------------------------------------------------

<!-- Upper half text -->
<div style="text-align: center; font-size: 0.7em; margin-bottom: 2vh;">
  <p>CKD often has no symptoms in early stages — making it a “silent disease".By the time symptoms appear, <strong>irreversible damage</strong> may have occurred.</p>
</div>

<!-- lower half text -->

:::{.columns}

::: {.column width="50%"}


In [ ]:
labels = ['Unaware of CKD', 'Aware of CKD']
sizes = [90, 10]
colors = ['#f4a261', '#2a9d8f']
fig, ax = plt.subplots(figsize=(4, 4), facecolor='none')
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_facecolor('none')   
plt.axis('equal')          
plt.tight_layout()
plt.show()

<p style="text-align: center; font-size: 0.50em; color: #444;">
<b>U.S. Perspective:</b><br>
9 out of 10 people with CKD are unaware they have it.<br>
<em>— American Kidney Fund</em>
</p>

:::

::: {.column width="50%"}


In [ ]:
labels = ['Terminal Stage', 'Early Stage']
sizes = [72.5, 27.5]
colors = ['#ba2f2f', '#87ceeb']

fig, ax = plt.subplots(figsize=(5, 5), facecolor='none')
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.set_facecolor('none')
plt.axis('equal')
plt.tight_layout()
plt.show()

<p style="text-align: center; font-size: 0.50em; color: #444;">
<b>Global Perspective:</b><br>
72.5% of patients were diagnosed at the terminal stage,  
and 75% had never heard of CKD before hospitalization.<br>
<em>— Brazzaville Hospital Study</em>
</p>

:::

:::


<div style="text-align: center; font-size: 0.6em; color: #666; margin-top: 1em;">
  Source: 
  <a href="https://www.kidneyfund.org" target="_blank">AKF</a> · 
  <a href="https://indexmedicus.afro.who.int/iah/fulltext/Chronic%20Renal%20Failure..." target="_blank">Brazzaville Study</a>
</div>


# Why Early Detection Matters { .center}

------------------------------------------------------------------------
That’s why we decided to build a **machine learning model** to assist in **early detection of CKD**, even during routine checkups — helping patients become aware of their symptoms and risks **before it’s too late**.

- Early detection enables:
  - Slowing or halting disease progression  
  - Improved quality of life  
  - Reduced need for dialysis or transplant
